In [1]:
import librosa
from librosa import display
import matplotlib
import matplotlib.pyplot as plt
import scipy
import os
from mido import MidiFile
import pandas as pd
import numpy as np
import csv
import multiprocessing as mp
import shutil
import sounddevice as sd
import guitarpro as gp
import madmom
import math
import io
import Transcriber
import scipy.io.wavfile
import PIL

In [2]:
def generateSpectogram(wavArray = None,filePath = None): ##can either give a Ndmpy array , or path to Png file to generate Spect
    
    if(filePath == None):
        recording1dimension = wavArray
        spectogram = librosa.stft(np.array(recording1dimension))
        mel = librosa.feature.melspectrogram(y=np.array(recording1dimension),sr=sample_rate,S=spectogram,n_fft=2048,hop_length=512,n_mels=128)
        mel_dB = librosa.amplitude_to_db(np.abs(mel)) 
       
    else:
        s,sr = librosa.load(filePath)
        spectogram = librosa.stft(s)
        mel = librosa.feature.melspectrogram(y=s,sr=sampling_rate,S=spectogram,n_fft=2048,hop_length=512,n_mels=128)
        mel_dB = librosa.amplitude_to_db(np.abs(mel)) 
        
    return mel_dB

# Load  .gp files with 1 guitar track into a data folder


In [18]:
def isSoloSong(filePath):
    try:
        songParsed = gp.parse(filePath)
        if(len(songParsed.tracks) == 1):
            return True
        else:
            return False
    except Exception:
        print(str(Exception)) 

In [ ]:
for symbol in os.listdir(r'C:\Users\Gebruiker\Desktop\DataBase Tabs GP3 format' ):
    for artist in os.listdir(r'C:\Users\Gebruiker\Desktop\DataBase Tabs GP3 format' + f'\\{symbol}'):
        for song in os.listdir(r'C:\Users\Gebruiker\Desktop\DataBase Tabs GP3 format' + f'\\{symbol}' + f'\\{artist}'):
            if(isSoloSong(r'C:\Users\Gebruiker\Desktop\DataBase Tabs GP3 format' + f'\\{symbol}' + f'\\{artist}' + f'\\{song}')):
                shutil.copy(src= r'C:\Users\Gebruiker\Desktop\DataBase Tabs GP3 format' + f'\\{symbol}' + f'\\{artist}' + f'\\{song}'
                            ,dst= r'C:\Users\Gebruiker\Desktop\PolyPhonic\data\training' + f'\\{song}.gp3')


# Transform gp files into .wav files

# Detecting Note onsets in .wav file and cutting them into 0.24s chunks

* we want snippets of 250ms , with a sampling rate of 44100 per second, this would mean an array of 11025 floating points.
 
recordingSnippetsList = [[snipetArray1],[snippetArray2],[...]]

# Transforming note snippets into spectogram



Value of NotesList past by reference to beatSlist -> upon changing former -> instance in latter also gets affected.

# Labeling spectogram files 

In [3]:
def __editSignalLength(signal,desired_length):
    
    if(len(signal) < desired_length):
        pad_amount = desired_length - len(signal)
        newSignal = np.pad(array=signal,pad_width= (0, pad_amount),mode='constant')
    else:
        newSignal = signal[:desired_length]
        
    return newSignal

In [4]:
def __generateOnsetFrames(pathGPsong,sampling_rate):
    beatList,songLength = Transcriber.transcribeGP3toTab(pathGPsong)
    onsetFrameValues = []
    onsetTimeValues = []
    onsetFrameValues.append(0)    ## avoiding substracting 0, resulting in negative number
    for index,row in enumerate(beatList[1:]):
        onsetTimeValues.append(row[0])
        onsetFrameValues.append(int((onsetTimeValues[index] * sampling_rate) - 220))
    return onsetFrameValues

There seems to be a delay of 10ms before note at time 0 starts playing (10ms * 44.1frames /ms) 441 frames 
-> shift every beat by 441 frames ?

In [5]:
def generateOnsetSpectograms1(pathAudio,durationTimeSecond,pathSave,GP3Path):
    samples,sampling_rate = librosa.load(pathAudio,sr=None,res_type='kaiser_fast')
    onsetFrames = __generateOnsetFrames(GP3Path,sampling_rate)
    durationFrame = int(sampling_rate* durationTimeSecond)

    for index,onset in enumerate(onsetFrames[:-1]):
        onset_padded = __editSignalLength(signal=samples[onset:onsetFrames[index + 1] - 320],desired_length=durationFrame) 
        plt.box(0)
        spectogram = librosa.stft(onset_padded,n_fft=2048,hop_length=512)
        mel_spectogram = librosa.feature.melspectrogram(sr=sampling_rate,S=spectogram,n_fft=2048,hop_length=512,n_mels=128)
        mel_dB = librosa.amplitude_to_db(np.abs(mel_spectogram)) 
        librosa.display.specshow(mel_dB,cmap='gray')
        plt.savefig(f'{pathSave}\\{index + 1}.png')
        plt.close()
    
    
    onset_padded = __editSignalLength(signal=samples[onsetFrames[-1]:(int(songLength * sampling_rate) - 320)],desired_length=durationFrame) 
    plt.box(0)
    spectogram = librosa.stft(onset_padded,n_fft=2048,hop_length=512)
    mel_spectogram = librosa.feature.melspectrogram(sr=sampling_rate,S=spectogram,n_fft=2048,hop_length=512,n_mels=128)
    mel_dB = librosa.amplitude_to_db(np.abs(mel_spectogram)) 
    librosa.display.specshow(mel_dB,cmap='gray')

    plt.savefig(f'{pathSave}\\{len(onsetFrames)}.png')
    plt.close()

In [44]:
def generateOnsetSpectograms2(pathAudio,durationTimeSecond,pathSave,GP3Path):
    samples,sampling_rate = librosa.load(pathAudio,sr=None,res_type='kaiser_fast')
    onsetFrames = __generateOnsetFrames(GP3Path,sampling_rate)
    durationFrame = int(sampling_rate* durationTimeSecond)

    for index,onset in enumerate(onsetFrames[:-1]):
        onset_padded = __editSignalLength(signal=samples[onset:onsetFrames[index + 1] - 320],desired_length=durationFrame) 
        plt.box(0)
        spectogram = librosa.stft(onset_padded,n_fft=2048,hop_length=512)
        mel_spectogram = librosa.feature.melspectrogram(sr=sampling_rate,S=spectogram,n_fft=2048,hop_length=512,n_mels=128)
        mel_dB = librosa.amplitude_to_db(np.abs(mel_spectogram)) 
        librosa.display.specshow(mel_dB)
        
        buffer = io.BytesIO()
        plt.savefig(buffer,format='png')
        plt.close()
        buffer.seek(0)
        img = PIL.Image.open(buffer,mode='r').convert('L')
        img.save(f'{pathSave}\\{index + 1}.png')  ## using save method from PIL rather than the broken Matplotlib    
    
    onset_padded = __editSignalLength(signal=samples[onsetFrames[-1]:(int(songLength * sampling_rate) - 320)],desired_length=durationFrame) 
    plt.box(0)
    spectogram = librosa.stft(onset_padded,n_fft=2048,hop_length=512)
    mel_spectogram = librosa.feature.melspectrogram(sr=sampling_rate,S=spectogram,n_fft=2048,hop_length=512,n_mels=128)
    mel_dB = librosa.amplitude_to_db(np.abs(mel_spectogram)) 
    librosa.display.specshow(mel_dB)
    
    buffer = io.BytesIO()
    plt.savefig(buffer,format='png')
    plt.close()
    buffer.seek(0)
    img = PIL.Image.open(buffer,mode='r').convert('L')
    img.save(f'{pathSave}\\{len(onsetFrames)}.png')  ## using save method from PIL rather than the broken Matplotlib 


In [8]:
def generateOnsetAmplitudes(pathAudio,durationTimeSecond,pathSave,GP3Path):
    samples,sampling_rate = librosa.load(pathAudio,sr=None,res_type='kaiser_fast')
    onsetFrames = __generateOnsetFrames(GP3Path,sampling_rate)
    durationFrame = int(sampling_rate * durationTimeSecond)
    
    
    for index,onset in enumerate(onsetFrames[:-1]):
        onset_padded = __editSignalLength(signal=samples[onset:onsetFrames[index + 1] - 320],desired_length=durationFrame) #
        fig = plt.figure()
        ax = fig.add_subplot()
        ax.axis('off')
        ax.plot(onset_padded)    
        plt.savefig(f'{pathSave}\\{index + 1}.png')
        plt.close()
    
    onset_padded = __editSignalLength(signal=samples[onsetFrames[-1]:(int(songLength * sampling_rate) - 320)],desired_length=durationFrame) #    
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.axis('off')
    ax.plot(onset_padded)    
    plt.savefig(f'{pathSave}\\{len(onsetFrames)}.png')
    plt.close()
## plot the figures between onset Frames 
## zero padding if the onsetbeat is shorter than 0.5s, cutting incase of the opposite 
                    

problem:  TuxGuitar GP3 to Wav converter doesn't convert the whole song sometimes? - > why 
The problem doesn't seem to be with our custom timer; after timing the song seperately, we found that
the TuxGuitar abruptly cuts the converted wav song. (Tarrega, Francisco - Capricho arabe (2).gp3.gp3) - measure 54

   possible solutions : 
1. Only generate beat Spectograms untill end of the wav Song, even if this is not the complete song.
2. Use a different converter
3. Further Diagnosis, mayby there's a meta variable that stops marks the end of the song that we're not aware of?
4. export to midi -> midi to wav?


* Note 1 Ok for some reason Mide File in MuseScore has 71, while the same GP3 file has only 62? 

    GP3 file has repeating barlines but musescore doesn't loop over it - > shorter song (*fixed , look in playback settings)

    Same GP3 file to midi doesn't have repeatBarliens at all?? 

    It seems to be a problem with the conversion not taking into account repeatingGroups of measure, though we still fall 12  seconds short of the real song

# EDIT , Problem lay in having to select RepeatLoop button before exporting.






* Note 2 MuseScore doesn't seem to correctly read all GuitarPro information (e.g tempo Measure)

* Note 3  Triplets are correctly calculated, we can already scrap this possibility

* Note 5 , guitarPro file and midi version of the same fileSong have different notations - > 
 Edit: It seems that even the same midi file prodocuces different different sheet notations, the problem must lie with the software interpreter intself(MuseScore)
 
* Note 6, the midi length seems to be the correct one 


# The problem may have to do with our algorithm note adapting the measure tempo on second loop + 

after correctly measuring we the song gets shortened by 3.6518 s making the total song 310s long (313.999 - 3.6518) * 


   *Midi stops at last note of measure ; rest measures get cut at end.
   
   
   
* Note 7  Transcribe to GP3 is songTime is longer than normal?

  Edit: There seems to be a problem with the TranscribeToGP3 triplet , sixplets. timing.
  
# TranscribeGP3File method doesn't factor triplet, sixplets?



To do :

* convert all single track GP3 files into Midi 
* convert Midi files to wav
* check if wav file match time with our algorithm 
* If so,cut into onsets and label
* make folders of songName , with beatImages in them.
* generate csv file


Note: gp3 file has 38 measures but only length 1:17? -> 12/8 time signature 180 bpm = 2 second? our algorithm calculates 4s


* Note 8 Why doesn't tupac repeatLoop , startRepeatGroup won't trigger   - > Perhaps because always repeatGroupLoop is checked at the next measure. When the last measure is the repeatClose; there is no next measure to trigger the repeatGroupLoop.


* Note 9 The dillema of constructing the algorithm to start repeatGroupLoop in the same iteration or the next.

* Note 10 perhaps the measureBeginRepeatGroup need not to be set to None after every loop; perhaps we can just leave it to the last openMeasure until a new one is triggered



* Find last non-empty beat index, + 1 

* Note 11 Librosa.load samples are float type

* Note 12, not all sheet beats get converted, midi only counts last non-empty TAB beats. making the Midi song slightly shorter sometimes than it really is.

* Audio snippets of 0.5s for each beat, with optionall padding or cutting

- cut audio into buckets between note and before next note. if bigger than 0.5s -> cut , if smaller -> zero padd
* Current problem , all images are same resolution, but different time length -> rescale all images to represent 0.5s?

  recurrent Neural network can classify based on variable length input images. Librosa.specschow() produces images of same       resolution but each pixel carying representing a different time. 
  
  -> rescale all images to a time domain of 0.5s, but don't zero pad or cut? Resulting in an image dimesion that properly 
  represent beat length, with each pixel carrying the same time length across images

![image.png](attachment:image.png)

 A beat can be transcribed on different positions on the fretboard, There are no "correct" positions, only better or worse   position in relation of difficulty of playing it. The difficulty of playing the generated TAB position for a beat depends on:


1. the relation of notes within the beat. 
2. the relation of the beat with the previous beats.
3. the relation of the beat with the upcoming beats.

with the influence of the beat decreasing the further away the beat is.


(samples,timestep,feature), timestep = (if timestep = samples , predict output at every sample (image))

return_sequence = False, where you only want the prediction at the last timestep e.g: Rating if a review is positive or negative when the whole review has been processed, rather than at the end of each word as in (return_Sequence=true).


the length of a pitch is correlated with the time axis , whereas the occurence and identification of a note only requires a very small time frame? - > what is this timeframe? - > 0.25s ?

In [14]:
def __secondToMinute(second):
    minute = 0  
    while(second - 60 >= 0):
        minute += 1
        second -= 60

    return str(f'{minute}:{round(second)}') 

In [11]:
for gp3File in os.scandir(r'D:\PolyPhonic\data\gp3'):
    beatList,songLength = Transcriber.transcribeGP3toTab(f'D:\\PolyPhonic\\data\\gp3\\{gp3File.name}')
    sample_rate,data = scipy.io.wavfile.read(f"D:\\PolyPhonic\\data\\wav\\{gp3File.name.rsplit('.',1)[0]}...wav")

    if(len(beatList[-1][1]) is 0):
        beatListReversed = beatList[::-1]
        beatListOnlyBeats = [row for row in beatListReversed if(len(row[1]) is not 0)]
        indexLastBeat = beatList.index(beatListOnlyBeats[0])
        generatedLengthSong = beatList[indexLastBeat + 1][0]
    else:
        generatedLengthSong = songLength
    print(f"{gp3File.name.rsplit('.',1)[0]} - lastBeat:{__secondToMinute(generatedLengthSong)} length:{__secondToMinute(len(data)//sample_rate)}")


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\PolyPhonic\\data\\wav\\1 God - Grace.wav'

In [15]:
def __checkValidityGeneratedSong(pathGP3,pathWav):   
    beatList,songLength = Transcriber.transcribeGP3toTab(pathGP3)
    sample_rate,data = scipy.io.wavfile.read(pathWav)
    #data,sample_rate = librosa.load(pathGP3,sr=None)
    if(len(beatList[-1][1]) is 0):
        beatListReversed = beatList[::-1]
        beatListOnlyBeats = [row for row in beatListReversed if(len(row[1]) is not 0)]
        indexLastBeat = beatList.index(beatListOnlyBeats[0])
        generatedLengthSong = beatList[indexLastBeat + 1][0]
    else:
        generatedLengthSong = songLength
    
    if((((len(data) / sample_rate) - generatedLengthSong) < 1) and (generatedLengthSong - (len(data) / sample_rate) < 1)):
        isValid = True
    else:
        isValid = False
    return isValid
    

In [12]:
for GP3File in os.scandir(r'D:\PolyPhonic\data\gp3'):
    pathWav = f"D:\\PolyPhonic\\data\\wav\\{GP3File.name.rsplit('.',1)[0]}..wav"
    pathSaveDir = f"D:\\PolyPhonic\\data\\training\\{GP3File.name.rsplit('.gp',2)[0]}"

    if(os.path.isdir(pathSaveDir)):
        print(pathSaveDir," Already Exists")
        
    elif(__checkValidityGeneratedSong(GP3File.path,pathWav)):
        os.mkdir(pathSaveDir)
        generateOnsetSpectograms2(pathAudio=pathWav,durationTimeSecond= 0.500,pathSave=pathSaveDir,GP3Path=GP3File.path)
        
    else:
        print(f"{GP3File.name} length not valid")

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\PolyPhonic\\data\\wav\\1 God - Grace..wav'

In [28]:
for song in os.scandir(r"D:\PolyPhonic\data\wav"):
    os.rename(src=song.path,dst=f"{song.path.rsplit('.gp',5)[0]}")
    
    #print(f"{song.path.rsplit('.gp',5)[0]}")
    

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'D:\\PolyPhonic\\data\\wav\\Black Sabbath - Bassically.gp4..wav' -> 'D:\\PolyPhonic\\data\\wav\\Black Sabbath - Bassically'

In [20]:
pathSong1 = r'D:\PolyPhonic\data\gp3\Albeniz, Isaac - Leyenda.gp3'
pathSong2 = r'D:\PolyPhonic\data\gp3\Mozart - Turkish March (2).gp4.'
pathSong3 = r'D:\PolyPhonic\data\gp3\Lizst, Franz - Liebesträume.gp3'
pathSong4 =  r'D:\PolyPhonic\data\gp3\Tarrega, Francisco - Capricho arabe (2).gp3'
pathSong5 = r'D:\PolyPhonic\data\gp3\8 Foot Sativa - Believer.gp4'
pathSong6 = r'D:\PolyPhonic\data\gp3\Bass Pandora - Fallen Angel.gp3'


beatList,songLength = Transcriber.transcribeGP3toTab(pathSong4)

for index,beat in enumerate(beatList):
    print(f'{index + 1} {beat}')
songLength

1 [0, [(4, 7), (5, 7), (6, 7)]]
2 [3.4615384615384612, [(1, 15)]]
3 [3.614599686028257, [(1, 12)]]
4 [3.7676609105180527, [(1, 10)]]
5 [3.9207221350078485, [(1, 9)]]
6 [4.073783359497645, [(1, 12)]]
7 [4.2268445839874405, [(1, 10)]]
8 [4.379905808477236, [(1, 6)]]
9 [4.532967032967032, [(1, 5)]]
10 [4.686028257456828, [(1, 8)]]
11 [4.8390894819466235, [(1, 6)]]
12 [4.992150706436419, [(2, 8)]]
13 [5.145211930926215, [(1, 0)]]
14 [5.298273155416011, [(1, 0)]]
15 [7.605965463108318, [(2, 3), (3, 3), (4, 5)]]
16 [8.471350078492934, [(2, 5)]]
17 [8.759811616954472, [(2, 2), (3, 2), (4, 2), (5, 0)]]
18 [12.221350078492934, [(4, 7), (5, 7), (6, 7)]]
19 [15.682888540031396, [(1, 15)]]
20 [15.835949764521192, [(1, 12)]]
21 [15.989010989010989, [(1, 10)]]
22 [16.142072213500786, [(1, 9)]]
23 [16.295133437990582, [(1, 12)]]
24 [16.44819466248038, [(1, 10)]]
25 [16.601255886970176, [(1, 6)]]
26 [16.754317111459972, [(1, 5)]]
27 [16.90737833594977, [(1, 8)]]
28 [17.060439560439566, [(1, 6)]]
29 [1

312.8560247883311

In [ ]:
#generateOnsetAmplitudes(r'D:\PolyPhonic\data\wav\Tarrega, Francisco - Capricho arabe (2).gp3..wav',GP3Path=pathSong4,pathSave=r'C:\Users\Gebruiker\Desktop\testFile2',durationTimeSecond=0.500)             
generateOnsetSpectograms2(r'D:\PolyPhonic\data\wav\Tarrega, Francisco - Capricho arabe (2).gp3..wav',GP3Path=pathSong4,pathSave=r'C:\Users\Gebruiker\Desktop\testFile',durationTimeSecond=0.500)                 


In [59]:
def checkMissingFiles():

    df = pd.read_csv(r"D:\PolyPhonic\data\training\labels.csv",encoding='cp1252')
    toDeleteFiles = []
    for index,filePath in enumerate(np.array(df["filename"])):
        if(os.path.exists(filePath) is False):
              print(filePath, index + 2)
              toDeleteFiles.append(filePath)

            
    return toDeleteFiles            
toDeleteFiles = checkMissingFiles()

D:\PolyPhonic\data\training\images\anonyme - au patit bonheur\83.png 22744
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\1.png 29701
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\2.png 29702
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\3.png 29703
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\4.png 29704
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\5.png 29705
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\6.png 29706
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\7.png 29707
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\8.png 29708
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\9.png 29709
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\10.png 29710
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\11.png 29711
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\12.png 29712
D:\PolyPhonic\data\training\images

D:\PolyPhonic\data\training\images\Ataris (The) - My Hotel Year\433.png 30760
D:\PolyPhonic\data\training\images\Atkins, Chet - A Little Mark Music\532.png 31292
D:\PolyPhonic\data\training\images\Atkins, Chet - All thumb\518.png 31810


In [102]:
def cutInvalidFilePaths(dfPath):
    df = pd.read_csv(dfPath,encoding='cp1252')
    df_arr_cut = []
    
    for row in np.array(df):
        if(os.path.exists(row[0])):
            df_arr_cut.append(row)
        else:
            print(row[0])
    
    return df_arr_cut

In [108]:
cutArray = cutInvalidFilePaths(r"D:\PolyPhonic\data\training\labels.csv")
pd.DataFrame(data=cutArray).to_csv(r"D:\PolyPhonic\data\training\labels2.csv",)

D:\PolyPhonic\data\training\images\anonyme - au patit bonheur\83.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\1.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\2.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\3.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\4.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\5.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\6.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\7.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\8.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\9.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\10.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\11.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\12.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\13.png
D:\PolyPhonic\data\training\images\At Vance

D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\408.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\409.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\410.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\411.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\412.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\413.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\414.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\415.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\416.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\417.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\418.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\419.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\420.png
D:\PolyPhonic\data\training\images\At Vance - Princess Of Ice\421.png
D:\PolyPhonic\data\t

D:\PolyPhonic\data\training\images\Atkins, Chet - A Little Mark Music\532.png
D:\PolyPhonic\data\training\images\Atkins, Chet - All thumb\518.png
